- Data su zo stranky www.soga.sk, jedna sa o vsetky aukcie od roku 2004 az po 2024, cca 100 aukcii a 21,000 drazenych diel

In [1]:
import sqlite3
import pandas as pd
import pandasql as ps
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import output_notebook

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('soga.db')

# Query to select data from the database
query = """
SELECT 
    UPPER(SUBSTR(autor, 1, INSTR(autor, '(') - 1)) AS autor,
    a.nazov, 
    vydrazene,
    CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) AS konecna_cena,
    CAST(REPLACE(vyvolavacia_cena, ' ', '') AS INTEGER) AS vyvolavacia_cena,
    rok,
    technika,
    datum,
    url_soga
FROM diela a 
LEFT JOIN aukcie b ON a.id_aukcie = b.id 
WHERE TRUE 
-- AND autor like '%FILKO%'
ORDER BY CAST(REPLACE(konecna_cena, ' ', '') AS INTEGER) DESC;
"""

# Load data into a Pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
df.head()

,autor,nazov,vydrazene,konecna_cena,vyvolavacia_cena,rok,technika,datum,url_soga
0,DOMENICHINO,Chlapec s dalmatínom,1,531103.0,663878.0,okolo 1615-25,olej na plátne,04. Október 2005,https://www.soga.sk/aukcie-obrazy-diela-umenie...
1,FULLA ĽUDOVÍT,Otčenáš,1,95000.0,NaN,1975,Olej na plátne (autorský rám),"21. Apríl 2015, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...
2,BENKA MARTIN,Katedrála Spišskej kapituly,1,65000.0,NaN,1942,Olej na plátne,"21. Apríl 2015, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...
3,ANTON JASUSCH,Divadelná šatňa (Zuzana a starci),1,48131.0,29875.0,okolo 1920 - 24,olej na plátne,28. Marec 2006,https://www.soga.sk/aukcie-obrazy-diela-umenie...
4,ANDREJ /ENDRE/ NEMEŠ /NEMES/,Ateliér,1,47135.0,27219.0,1937,olej na plátne,11. Marec 2008,https://www.soga.sk/aukcie-obrazy-diela-umenie...


# Priprava dat

In [3]:
from unidecode import unidecode  # Import the unidecode function
import re


# Custom function to standardize and reorder names
def standardize_and_order_name(name):
    # Convert accented characters to ASCII equivalents
    name = unidecode(name)
    name= name.upper()

    # Split the name into parts
    name_parts = name.split()

    # Reorder the name parts alphabetically
    standardized_name = ' '.join(sorted(name_parts))
    
    

    return standardized_name


def extract_first_year(date_string):
    # Regular expression pattern to match a year
    year_pattern = re.compile(r'\b\d{4}\b')

    # Find the first match in the string
    match = year_pattern.search(date_string)

    # Extract and return the matched year
    if match:
        return int(match.group())

    return None  # Return None if no year is found


In [4]:
df['autor']=df['autor'].apply(standardize_and_order_name)

In [5]:
df['olej']=df['technika'].apply(lambda x: 'olej' in x.lower())

In [6]:
df['rok_vytvorenia']=df['rok'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)
df['rok_drazby']=df['datum'].apply(lambda x: extract_first_year(x)).fillna(-1).astype(int).replace(-1,None)

In [7]:
df.head()

,autor,nazov,vydrazene,konecna_cena,vyvolavacia_cena,rok,technika,datum,url_soga,olej,rok_vytvorenia,rok_drazby
0,DOMENICHINO,Chlapec s dalmatínom,1,531103.0,663878.0,okolo 1615-25,olej na plátne,04. Október 2005,https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1615,2005
1,FULLA LUDOVIT,Otčenáš,1,95000.0,NaN,1975,Olej na plátne (autorský rám),"21. Apríl 2015, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1975,2015
2,BENKA MARTIN,Katedrála Spišskej kapituly,1,65000.0,NaN,1942,Olej na plátne,"21. Apríl 2015, 19:00",https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1942,2015
3,ANTON JASUSCH,Divadelná šatňa (Zuzana a starci),1,48131.0,29875.0,okolo 1920 - 24,olej na plátne,28. Marec 2006,https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1920,2006
4,/ENDRE/ /NEMES/ ANDREJ NEMES,Ateliér,1,47135.0,27219.0,1937,olej na plátne,11. Marec 2008,https://www.soga.sk/aukcie-obrazy-diela-umenie...,True,1937,2008


# Analyza

## Top 50

### Top 50 najdrazsich autorov podla priemernej ceny

In [8]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
--and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

,autor,pocet_vydrazenych_od_2004,priemerna_vydrazena_cena_EUR,minimalna_vydrazena_cena_EUR,maximalna_vydrazena_cena_EUR
0,FULLA LUDOVIT,7,17761.0,190.0,95000.0
1,BENKA MARTIN,19,15903.0,1000.0,65000.0
2,LADISLAV MEDNYANSZKY,26,10856.0,266.0,35000.0
3,ANTON JASUSCH,16,8769.0,1494.0,48131.0
4,HALA JAN,14,7143.0,398.0,38000.0
5,ALEXANDER BAZOVSKY MILOS,11,6168.0,1394.0,20000.0
6,JOZEF MOUSSON TEODOR,7,4074.0,597.0,15933.0
7,GUSTAV MALLY,7,3988.0,398.0,13300.0
8,CSORDAK LUDOVIT,6,3926.0,350.0,8000.0
9,LALUHA MILAN,9,3491.0,199.0,15700.0


### Top 50 najdrazsich autorov - oleje

In [9]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_vydrazenych_od_2004, 
ROUND(avg(konecna_cena)) as priemerna_vydrazena_cena_EUR,
min(konecna_cena)as minimalna_vydrazena_cena_EUR,
max(konecna_cena)as maximalna_vydrazena_cena_EUR
--sum(konecna_cena)as celkovo_vydrazene_EUR
from df 
where True 
and konecna_cena is not null 
and olej
group by 1 
having count(*)>5
order by 3
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]

,autor,pocet_vydrazenych_od_2004,priemerna_vydrazena_cena_EUR,minimalna_vydrazena_cena_EUR,maximalna_vydrazena_cena_EUR
0,BENKA MARTIN,16,18555.0,1992.0,65000.0
1,LADISLAV MEDNYANSZKY,16,16216.0,2987.0,35000.0
2,HALA JAN,9,10550.0,1693.0,38000.0
3,ANTON JASUSCH,14,9507.0,1494.0,48131.0
4,IMRO WEINER-KRAL,6,6154.0,3319.0,14937.0
5,GUSTAV MALLY,6,4587.0,1228.0,13300.0
6,KOMPANEK VLADIMIR,13,2565.0,730.0,11500.0
7,KRIZAN LUDOVIT,6,2197.0,800.0,4500.0
8,ERNEST ZMETAK,15,1971.0,398.0,8298.0
9,MARIA MEDVECKA,17,1929.0,564.0,10000.0


### Top 50 najdrazsich diel

In [10]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,DOMENICHINO,Chlapec s dalmatínom,531103.0,1615,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/domenichino-domenico-zampieri-chlapec-s-dalmatinom-9260
1,FULLA LUDOVIT,Otčenáš,95000.0,1975,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/fulla-ludovit-otcenas-52930
2,BENKA MARTIN,Katedrála Spišskej kapituly,65000.0,1942,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-katedrala-spisskej-kapituly-52913
3,ANTON JASUSCH,Divadelná šatňa (Zuzana a starci),48131.0,1920,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/63-jarna-aukcia-vytvarnych-diel/anton-jasusch-divadelna-satna-zuzana-a-starci-11076
4,/ENDRE/ /NEMES/ ANDREJ NEMES,Ateliér,47135.0,1937,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/78-jarna-aukcia-vytvarnych-diel/andrej-endre-nemes-nemes-atelier-16832
5,JOSEF MATEJ NAVRATIL,Panská spoločnosť,44812.0,1840,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/63-jarna-aukcia-vytvarnych-diel/josef-matej-navratil-panska-spolocnost-10953
6,BENKA MARTIN,Pod Chočom,42000.0,1921,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-pod-chocom-52918
7,DOMINGUEZ OSCAR,Galský kohút,40497.0,1949,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/61-zimna-aukcia-vytvarnych-diel/oscar-dominguez-galsky-kohut-9898
8,,Kabinet,39833.0,1690,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/62-aukcia-umeleckeho-remesla-a-starozitnosti/kabinet-10390
9,HALA JAN,Kosec,38000.0,1948,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/hala-jan-kosec-51131


### Top 50 najdrazsich diel - nie oleje

In [11]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
and not olej
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,,Kabinet,39833.0,1690,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/62-aukcia-umeleckeho-remesla-a-starozitnosti/kabinet-10390
1,JANKOVIC JOZEF,Svedectvo V.,33194.0,1967,2007,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/75-aukcia-sucasneho-slovenskeho-umenia/jozef-jankovic-svedectvo-v-14943
2,,Klaňanie troch kráľov,26555.0,None,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/62-aukcia-umeleckeho-remesla-a-starozitnosti/neznamy-nemecky-maliar-klananie-troch-kralov-10388
3,,Busta pápeža (podľa Gianlorenza Berniniho),24000.0,None,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/118-jesenna-aukcia-vytvarnych-diel/neznamy-taliansky-sochar-busta-papeza-podla-gianlorenza-berniniho-51592
4,ARMAND SCHONBERGER,Čítajúca,21576.0,1930,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/67-jesenna-aukcia-vytvarnych-diel/armand-schonberger-citajuca-12353
5,,Barokový tabernákulum,21244.0,1720,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/62-aukcia-umeleckeho-remesla-a-starozitnosti/barokovy-tabernakulum-10406
6,,Gotická madona,19252.0,None,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/62-aukcia-umeleckeho-remesla-a-starozitnosti/goticka-madona-10356
7,CYPRIAN MAJERNIK,Hliadka,18000.0,1943,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/majernik-cyprian-hliadka-52923
8,,Svätá Barbora,15435.0,1650,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/62-aukcia-umeleckeho-remesla-a-starozitnosti/svata-barbora-10402
9,GUDERNA LADISLAV,Hlava ženy (Melanchólia),15000.0,1946,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/guderna-ladislav-hlava-zeny-melancholia-52943


### Top 50 najdrazsich diel - Pasteka

In [12]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%PASTEKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,MILAN PASTEKA,Kompozícia s červenou,4979.0,1989,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/61-zimna-aukcia-vytvarnych-diel/milan-pasteka-kompozicia-s-cervenou-9871
1,MILAN PASTEKA,Zátišie,2921.0,1974,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/milan-pasteka-zatisie-9179
2,MILAN PASTEKA,V čakárni,2622.0,1982,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/61-zimna-aukcia-vytvarnych-diel/milan-pasteka-v-cakarni-9873
3,MILAN PASTEKA,Na stoličke,1500.0,1988,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/pasteka-milan-na-stolicke-51329
4,MILAN PASTEKA,Odcudzenie,1328.0,1980,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/milan-pasteka-odcudzenie-6098
5,MILAN PASTEKA,Žena s veľkým vtákom,1062.0,1977,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/milan-pasteka-zena-s-velkym-vtakom-9178
6,MILAN PASTEKA,Torzá,996.0,1986,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/milan-pasteka-torza-6097


### Top 50 najdrazsich diel - Binder

In [13]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BINDER %'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,BINDER ERIK,"Ja somár, načo som sa snažil?!",5000.0,2014,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/binder-erik-ja-somar-naco-som-sa-snazil-52982
1,BINDER ERIK,"O Elektrike (A tak si volám taxi, okolo lietajú smsky a faxy)",3700.0,2013,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/binder-erik-o-elektrike-a-tak-si-volam-taxi-okolo-lietaju-smsky-a-faxy-51356
2,BINDER ERIK,"Už chýba iba... , z cyklu Mentálne stavy",1700.0,2014,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/118-jesenna-aukcia-vytvarnych-diel/binder-erik-uz-chyba-iba-z-cyklu-mentalne-stavy-51540
3,BINDER ERIK,"Ty, had!",66.0,2004,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/50-mimoriadna-aukcia-sucasneho-slovenskeho-umenia/erik-binder-ty-had-5598
4,BINDER ERIK,Choď a zabi to! 01,66.0,2004,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/50-mimoriadna-aukcia-sucasneho-slovenskeho-umenia/erik-binder-chod-a-zabi-to-01-5596


### Top 50 najdrazsich diel - Filko

In [14]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILKO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FILKO STANO,Figurálna kompozícia,11200.0,1958,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/filko-stano-figuralna-kompozicia-51328
1,FILKO STANO,Slovak Šeherezáda – Šiška,5200.0,1978,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/filko-stano-slovak-seherezada-siska-52958
2,FILKO STANO,Subjekt II.,4149.0,1963,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/stano-filko-subjekt-ii-9160
3,FILKO STANO,Narodenie,1195.0,1960,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/50-mimoriadna-aukcia-sucasneho-slovenskeho-umenia/stano-filko-narodenie-5641
4,FILKO STANISLAV,Subjekt IV.,531.0,1960,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/stanislav-filko-subjekt-iv-6120
5,FILKO STANO,Historická civilizácia na zemi,266.0,1967,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/67-jesenna-aukcia-vytvarnych-diel/stano-filko-historicka-civilizacia-na-zemi-12200
6,FILKO STANKO,Venuša - Dedinčanka,100.0,1950,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/stanko-filko-venusa-dedincanka-9159


### Top 50 najdrazsich diel - Laluha

In [15]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%LALUHA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,LALUHA MILAN,Akt,15700.0,1970,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/laluha-milan-akt-52950
1,LALUHA MILAN,Chlap a žena,5809.0,1966,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/63-jarna-aukcia-vytvarnych-diel/milan-laluha-chlap-a-zena-10933
2,LALUHA MILAN,Žena,2158.0,1970,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/61-zimna-aukcia-vytvarnych-diel/milan-laluha-zena-9878
3,LALUHA MILAN,Ležiaci akt,2058.0,1964,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/milan-laluha-leziaci-akt-6102
4,LALUHA MILAN,Žena s nošou,2058.0,1968,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/milan-laluha-zena-s-nosou-9185
5,LALUHA MILAN,Tetka,1826.0,None,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/milan-laluha-tetka-6103
6,LALUHA MILAN,Stromy,1261.0,1963,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/milan-laluha-stromy-9186
7,LALUHA MILAN,Dve ženy s hrabľami,350.0,1980,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/laluha-milan-dve-zeny-s-hrablami-51323
8,LALUHA MILAN,Starci,199.0,1999,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/milan-laluha-starci-9187


### Top 50 najdrazsich diel - Benka

In [16]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%BENKA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,BENKA MARTIN,Katedrála Spišskej kapituly,65000.0,1942,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-katedrala-spisskej-kapituly-52913
1,BENKA MARTIN,Pod Chočom,42000.0,1921,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-pod-chocom-52918
2,BENKA MARTIN,Ranné hmly nad Liskovou,36000.0,1921,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/121-jarna-aukcia-vytvarnych-diel/benka-martin-ranne-hmly-nad-liskovou-52504
3,BENKA MARTIN,Z Liptova (Ružomberok),25000.0,1935,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-z-liptova-ruzomberok-52906
4,BENKA MARTIN,Juhočeská krajina,22738.0,1914,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/83-zimna-aukcia-vytvarnych-diel-a-starozitnosti/martin-benka-juhoceska-krajina-18633
5,BENKA MARTIN,Na brehu rieky Revúcej - Liptov,22406.0,1941,2006,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/67-jesenna-aukcia-vytvarnych-diel/martin-benka-na-brehu-rieky-revucej-liptov-12392
6,BENKA MARTIN,Brod cez rieku Váh,15000.0,1921,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/benka-martin-brod-cez-rieku-vah-52921
7,BENKA MARTIN,Terchovianky,14937.0,1935,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/martin-benka-terchovianky-6382
8,BENKA MARTIN,Žena v čepci,13444.0,1938,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/61-zimna-aukcia-vytvarnych-diel/martin-benka-zena-v-cepci-10044
9,BENKA MARTIN,Na horskej lúke,12116.0,1925,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/61-zimna-aukcia-vytvarnych-diel/martin-benka-na-horskej-luke-10045


### Top 50 najdrazsich diel - Fulla

In [17]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FULLA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,FULLA LUDOVIT,Otčenáš,95000.0,1975,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/122-vecerna-aukcia/fulla-ludovit-otcenas-52930
1,FULLA LUDOVIT,Kytica tulipánov,26555.0,1930,2008,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/83-zimna-aukcia-vytvarnych-diel-a-starozitnosti/ludovit-fulla-kytica-tulipanov-18625
2,FULLA LUDOVIT,Dedina pri Ružomberku,929.0,1960,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/61-zimna-aukcia-vytvarnych-diel/ludovit-fulla-dedina-pri-ruzomberku-10027
3,FULLA LUDOVIT,Zvestovanie,700.0,1948,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/121-jarna-aukcia-vytvarnych-diel/fulla-ludovit-zvestovanie-52527
4,FULLA LUDOVIT,Narodenie,600.0,1948,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/121-jarna-aukcia-vytvarnych-diel/fulla-ludovit-narodenie-52526
5,FULLA LUDOVIT,Ždiaranka,350.0,1947,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/fulla-ludovit-zdiaranka-51230
6,FULLA LUDOVIT,PF 1979,190.0,1978,2015,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/121-jarna-aukcia-vytvarnych-diel/fulla-ludovit-pf-1979-52528


### Top 50 najdrazsich diel - Klimo

In [18]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%KLIMO%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,ALOJZ KLIMO,Okná,1992.0,1966,2005,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/59-jesenna-aukcia-vytvarnych-diel/alojz-klimo-okna-9172
1,ALOJZ KLIMO,Rozdelenie,564.0,1993,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/alojz-klimo-rozdelenie-6119
2,ALOJZ KLIMO,Okno,320.0,1968,2014,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/117-letna-aukcia-vytvarnych-diel-a-starozitnosti/klimo-alojz-okno-51345


### Top 50 najdrazsich diel - Roller

In [19]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%ROLLER%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga


### Top 50 najdrazsich diel - Fila

In [20]:
# SQL-like query
query = """
select 
autor,
nazov,
konecna_cena,
cast(rok_vytvorenia as varchar) rok_vytvorenia,
rok_drazby,
url_soga
from df 
where True 
and konecna_cena is not null 
--and not olej
and autor like '%FILA%'
order by konecna_cena
DESC 
"""

# Apply the query using pandasql
result_df = ps.sqldf(query, locals())


# Display the result DataFrame
HTML(result_df[0:50].to_html(render_links=True, escape=False))

,autor,nazov,konecna_cena,rok_vytvorenia,rok_drazby,url_soga
0,EMIL FILA,Kubistická kompozícia,830.0,None,2004,https://www.soga.sk/aukcie-obrazy-diela-umenie-starozitnosti/aukcie/51-jesenna-aukcia-vytvarnych-diel/emil-fila-kubisticka-kompozicia-6195


## Najvyssi narast a pokles cien

### Najvyssi narast

In [21]:
from scipy.stats import linregress
pd.set_option('display.max_colwidth', None)

# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].agg(['count', 'mean']).reset_index()
df_avg = df_avg[df_avg['count']>=3][['autor', 'rok_drazby', 'mean']]

# Filter authors with at least 50 rows
author_counts = df_avg['autor'].value_counts()
selected_authors = author_counts[author_counts >= 5].index
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Calculate the slope for each author
slopes = []

for author in selected_authors:
    author_data = df_avg_filtered[df_avg_filtered['autor'] == author]
    slope, _, _, _, _ = linregress(author_data['rok_drazby'], author_data['mean'])
    slopes.append({'autor': author, 'priemerny_rocny_narast_v_cene': slope})

# Create a DataFrame for the slopes
slopes_df = pd.DataFrame(slopes)

# Display the slopes
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=False).reset_index(drop=True)[0:20]

ModuleNotFoundError: No module named 'scipy'

In [ ]:
slopes_df[slopes_df['priemerny_rocny_narast_v_cene'].notna()].sort_values('priemerny_rocny_narast_v_cene', ascending=True).reset_index(drop=True)[0:20]

In [ ]:
# Calculate the average price for each name in each year
df_avg = df.groupby(['autor', 'rok_drazby'])['konecna_cena'].mean().reset_index()

# Filter authors with at least 50 rows
selected_authors = ['MILAN PASTEKA', 'FILKO STANO', 'LALUHA MILAN']
df_avg_filtered = df_avg[df_avg['autor'].isin(selected_authors)]

# Create Bokeh ColumnDataSource
source = ColumnDataSource(df_avg_filtered)

# Plotting
output_notebook()
p = figure(plot_width=800, plot_height=400, title='Average Price Trends Over Time for Each Name',
           x_axis_label='Year', y_axis_label='Average Price')

# Add a line for each autor
for color, autor in zip(Category10[6], df_avg_filtered['autor'].unique()):
    autor_data = df_avg_filtered[df_avg_filtered['autor'] == autor]
    autor_source = ColumnDataSource(autor_data)
    p.line(x='rok_drazby', y='konecna_cena', source=autor_source, line_width=2, color=color, legend_label=autor)
    p.circle(x='rok_drazby', y='konecna_cena', source=autor_source, size=8, color=color, legend_label=autor)

p.legend.location = 'top_left'
show(p)

## Najvyssie percento vydrazenych

In [ ]:
# SQL-like query
query = """
select 
autor, 
count(*) pocet_drazenych_od_2004, 
sum(vydrazene) as pocet_vydrazenych_od_2004,
round(sum(vydrazene)*100.0/count(*),2) as percento_vydrazenych
from df 
where True 
--and konecna_cena is not null 
--and olej
group by 1 
having count(*)>50
order by 4
DESC 
"""


# Apply the query using pandasql
result_df = ps.sqldf(query, locals())

# Display the result DataFrame
result_df[0:50]